In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score
from  sklearn.ensemble import RandomForestRegressor
plt.style.use('fivethirtyeight')
%matplotlib inline


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Reading the data

In [ ]:
train = pd.read_csv('/data/train.csv')
test = pd.read_csv('/data/test.csv')
sample_sub = pd.read_csv('/data/sample_submission.csv')
characters = pd.read_csv('/data/characters.csv')
locations = pd.read_csv('/data/locations.csv')
script_lines = pd.read_csv('/data/script_lines.csv')

In [ ]:
train.head()

In [ ]:
characters.head()

In [ ]:
locations.head()

In [ ]:
script_lines.head()

# EDA & FE

First, let's check a few distributions.

In [ ]:
# Target variable
train['imdb_score'].hist()
plt.title('Discribution of IMDB scores.')
plt.show()

The distribution looks close to normal with very small amount of episodes with low ratings.

Let's check if there is correlation between number of viewers of the episode and number of votes with target variable.

In [ ]:
train[['imdb_votes', 'us_viewers_in_millions', 'imdb_score']].corr()

We have correlations scores higher than 0.6. Let's try to create some features using this knowledge.

In [ ]:
# Checking top viewed episodes.
train.sort_values('us_viewers_in_millions', ascending=False).head(20)

Few observations that we can transform into features:
* Old episodes have more viewers that newer ones.
* Some episodes contain character name in the title. Create some features out of it?
* "Treehouse of Horror" is the title for special episodes, it makes sense to create feature using this info.

In [ ]:
# Creating "Treehouse of Horror" feature for both train and test.
train['is_halloween_special'] = train['title'].apply(lambda x: 1 if 'treehouse of horror' in x.lower() else 0)
test['is_halloween_special'] = test['title'].apply(lambda x: 1 if 'treehouse of horror' in x.lower() else 0)

Now let's check the contents of script lines and see if we can extract some meaningful features from it.

In [ ]:
script_lines.head()

We have the character id, so some ideas here:
* Number of lines for character per episode
* Some statistics over the spoken lines

In [ ]:
# Creating 'Lisa index' - number of Lisa's lines per episode
lisa_index = script_lines.loc[script_lines['character_id']==9,:].groupby(['episode_id'])['spoken_words'].count()

In [ ]:
# Calculating also total number of lines per episode.
lines_per_episode = script_lines.groupby(['episode_id'])['spoken_words'].count()

In [ ]:
# Combining new features with train and test data.
train['lisa_index'] = train['id'].map(lisa_index).fillna(0)
test['lisa_index'] = test['id'].map(lisa_index).fillna(0)

train['n_lines'] = train['id'].map(lines_per_episode).fillna(0)
test['n_lines'] = test['id'].map(lines_per_episode).fillna(0)